In [7]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [8]:
import pandas as pd

# Charger le fichier CSV téléchargé dans un DataFrame pour l'entraînement
file_name_train = list(uploaded.keys())[0]  # Récupérer le nom du fichier d'entraînement téléchargé
data_train = pd.read_csv(file_name_train)

# Charger le fichier CSV téléchargé dans un DataFrame pour les tests
file_name_test = list(uploaded.keys())[1]  # Récupérer le nom du fichier de test téléchargé
data_test = pd.read_csv(file_name_test)

# Afficher les premières lignes des datasets
print("Premières lignes du dataset d'entraînement:")
print(data_train.head())

print("\nPremières lignes du dataset de test:")
print(data_test.head())



Premières lignes du dataset d'entraînement:
           id                                            premise  \
0  c6d58c3f69  بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...   
1  cefcc82292                             هذا هو ما تم نصحنا به.   
2  e98005252c  et cela est en grande partie dû au fait que le...   
3  58518c10ba                   与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp   
4  c32b0d16df                              Она все еще была там.   

                                          hypothesis lang_abv language  
0  کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی...       ur     Urdu  
1  عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...       ar   Arabic  
2                             Les mères se droguent.       fr   French  
3                            IMA与其他组织合作，因为它们都依靠共享资金。       zh  Chinese  
4     Мы думали, что она ушла, однако, она осталась.       ru  Russian  

Premières lignes du dataset de test:
           id                                            premise  \
0  

In [ ]:
print(data_train.columns)
print(data_test.columns)

Index(['id', 'premise', 'hypothesis', 'lang_abv', 'language'], dtype='object')
Index(['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label'], dtype='object')


In [ ]:
from transformers import BertTokenizer, XLMRobertaTokenizer
import pandas as pd

# Charger les tokenizers BERT et XLM-RoBERTa
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
xlm_roberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

# Fonction pour la tokenisation avec BertTokenizer
def tokenize_pair_bert_batch(premises, hypotheses):
    encoding = bert_tokenizer.batch_encode_plus(
        list(zip(premises, hypotheses)),  # Combiner les deux colonnes en une seule liste de paires
        add_special_tokens=True,   # Ajouter les tokens spéciaux [CLS] et [SEP]
        padding='max_length',      # Ajouter un padding si nécessaire
        max_length=15,             # Limiter la longueur à 15 tokens
        truncation=True,           # Tronquer si la phrase est trop longue
        return_tensors='pt'        # Retourner les tensors pour PyTorch
    )
    return encoding

# Fonction pour la tokenisation avec XLM-RoBERTa
def tokenize_pair_xlm_batch(premises, hypotheses):
    encoding = xlm_roberta_tokenizer.batch_encode_plus(
        list(zip(premises, hypotheses)),  # Combiner les deux colonnes en une seule liste de paires
        add_special_tokens=True,   # Ajouter les tokens spéciaux <s> et </s>
        padding='max_length',      # Ajouter un padding si nécessaire
        max_length=15,             # Limiter la longueur à 15 tokens
        truncation=True,           # Tronquer si la phrase est trop longue
        return_tensors='pt'        # Retourner les tensors pour PyTorch
    )
    return encoding

# Tokenisation par lots pour le dataset d'entraînement
premises_train = data_train['premise'].tolist()
hypotheses_train = data_train['hypothesis'].tolist()

encoded_data_train_bert = tokenize_pair_bert_batch(premises_train, hypotheses_train)
encoded_data_train_xlm = tokenize_pair_xlm_batch(premises_train, hypotheses_train)

# Tokenisation par lots pour le dataset de test
premises_test = data_test['premise'].tolist()
hypotheses_test = data_test['hypothesis'].tolist()

encoded_data_test_bert = tokenize_pair_bert_batch(premises_test, hypotheses_test)
encoded_data_test_xlm = tokenize_pair_xlm_batch(premises_test, hypotheses_test)

# Exemple d'affichage des résultats pour la première tokenisation
print("Premier encodage BERT pour le dataset d'entraînement:", encoded_data_train_bert['input_ids'][0])
print("Premier encodage XLM-RoBERTa pour le dataset d'entraînement:", encoded_data_train_xlm['input_ids'][0])

print("\nPremier encodage BERT pour le dataset de test:", encoded_data_test_bert['input_ids'][0])
print("Premier encodage XLM-RoBERTa pour le dataset de test:", encoded_data_test_xlm['input_ids'][0])


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflow

In [ ]:
print(train_data.columns)


Index(['id', 'premise', 'hypothesis', 'lang_abv', 'language'], dtype='object')


In [9]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Création d'une colonne artificielle pour la stratification (ex: longueur des phrases)
data_train['stratify_group'] = data_train['premise'].apply(lambda x: len(x.split())) // 5

# Extraire les groupes pour la stratification
stratify_labels = data_train['stratify_group'].values

# Initialiser StratifiedKFold avec 5 plis
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Stocker les indices des plis
train_indices_list = []
val_indices_list = []

# Générer les plis
for train_index, val_index in kf.split(data_train, stratify_labels):
    train_indices_list.append(train_index)
    val_indices_list.append(val_index)

# Supprimer la colonne artificielle après la stratification
data_train.drop(columns=['stratify_group'], inplace=True)

# Affichage des indices du premier pli
print("Indices du premier pli (train) :", train_indices_list[0])
print("Indices du premier pli (validation) :", val_indices_list[0])


Indices du premier pli (train) : [   0    1    2 ... 5191 5192 5193]
Indices du premier pli (validation) : [   3    5    9 ... 5184 5187 5194]


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
